<a href="https://colab.research.google.com/github/manishiitg/ML_Experiments/blob/master/autoencoder/pytorch_autoencoder_for_nlp_with_lstm_%2B_cnn.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

currently stuck on this as unable to do this properly that is getting output. maybe explore cnn etc.

 **mainly problem is the model predictions unable to convert them back to sentense. i think mainly doing some silly mistakes need to rethink stuff etc and approach with a fresh mind**

In [0]:
import torch
from torchtext import data
from torchtext import datasets

SEED = 1234

torch.manual_seed(SEED)
torch.backends.cudnn.deterministic = True

TEXT = data.Field(tokenize = 'spacy', include_lengths = True)
LABEL = data.LabelField(dtype = torch.float)







from sklearn.datasets import fetch_20newsgroups

categories = [ 'alt.atheism', 'comp.graphics','rec.autos','sci.crypt','talk.politics.guns']
news = fetch_20newsgroups(subset="train", categories=categories)

max_limit = 4000

targets = news["target"][:max_limit]
filenames = news["filenames"][:max_limit]
news = news["data"][:max_limit]






from torchtext.data import Dataset

class NewGroupDataset(Dataset):
    def __init__(self, text_field, label_field, **kwargs):
        fields = [("Description", text_field), ("Category", label_field)]
        examples = []
        
        for idx, n in enumerate(news):
          examples.append(data.Example.fromlist([n, targets[idx]], fields))
        super().__init__(examples, fields, **kwargs)

    @staticmethod
    def sort_key(ex): return len(ex.Description)
    
    @classmethod
    def splits(cls, text_field, label_field, root='.data',
               train='train', test='test', **kwargs):
        return super().splits(
            root, text_field=text_field, label_field=label_field,
            train=train, validation=None, test=test, **kwargs)




full_data_set = NewGroupDataset(TEXT, LABEL)
print("full dataset length" , len(full_data_set))



from torchtext.vocab import Vectors, GloVe

vectors = GloVe(name='6B', dim=300)

TEXT.build_vocab(full_data_set, vectors=vectors)
LABEL.build_vocab(full_data_set)





BATCH_SIZE = 64

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

full_data_iterator = data.BucketIterator(full_data_set, 
    batch_size = BATCH_SIZE,
    sort_within_batch = True,
    device = device)

full dataset length 2799


In [0]:
# import random

# train_data, test_data = NewGroupDataset.splits(TEXT, LABEL)
# train_data, valid_data = train_data.split(random_state = random.seed(SEED))

# print(f'Number of training examples: {len(train_data)}')
# print(f'Number of validation examples: {len(valid_data)}')
# print(f'Number of testing examples: {len(test_data)}')

In [0]:
# print(TEXT.vocab.stoi[200])
# print(TEXT.vocab.itos[100])

print(TEXT.vocab.vectors.shape)


torch.Size([53168, 300])


In [0]:
# def find_distance():
cat=vectors[stoi['cat']]

dist = ((vectors-cat)**2).sum(1)
dist_sorted, dist_sorted_indexes = dist.sort(0)

print([vectors.itos[i] for i in dist_sorted_indexes[0:10]])

NameError: ignored

In [0]:
import torch.nn as nn
import torch.nn.functional as F
import numpy as np

class LSTMAutoencoder(nn.Module):
    def __init__(self, vocab_size, embedding_dim, hidden_dim, latent_dim, n_layers, 
                 bidirectional, dropout, pad_idx, weights):
        
        super().__init__()

        # self.hidden_dim = hidden_dim
        self.latent_dim = latent_dim
        
        self.rnn_encoder = nn.LSTM(embedding_dim, 
                           latent_dim, 
                           num_layers=1, 
                           bidirectional=False)
        
        # self.fc_encoder = nn.Linear(hidden_dim * 2, latent_dim)

        # self.fc_decoder = nn.Linear(latent_dim, hidden_dim * 2)

        self.rnn_decoder = nn.LSTM(latent_dim,embedding_dim, 
                           num_layers=1, 
                           bidirectional=False)
        
        # self.dropout = nn.Dropout(dropout)
        
    def forward(self, text):
        
        # print(text.shape, "input text shape")
        # embedded = self.dropout(self.embedding(text))
        
        _, (hidden, _) = self.rnn_encoder(text)
        # print(hidden.shape , "hidden shap after encoder rnn")
        

        # hidden = self.dropout(torch.cat((hidden[-2,:,:], hidden[-1,:,:]), dim = 1))

        # hidden = torch.cat((hidden[-2,:,:], hidden[-1,:,:]), dim = 1)
        # print(hidden.shape, "hidden shape after concat")

        # hidden = hidden.unsqueeze(0)
                
        #hidden = [batch size, hid dim * num directions]

        # encoded = F.relu(self.fc_encoder(hidden))
        # print(encoded.shape , "encoder output shape")

        # decoded_fc = F.relu(self.fc_decoder(encoded))
        # print(decoded_fc.shape)

        encoded_repeat = hidden.repeat(len(text),1,1)
        # print(encoded_repeat.shape, "encode repeat shape")

        decoded, _ = self.rnn_decoder(encoded_repeat)
        

        # print(decoded.shape, " decoded output")
            
        return decoded

INPUT_DIM = len(TEXT.vocab)
EMBEDDING_DIM = 300
HIDDEN_DIM = 256
LATENT_DIM = 10
N_LAYERS = 2
BIDIRECTIONAL = True
DROPOUT = 0
PAD_IDX = TEXT.vocab.stoi[TEXT.pad_token]

pretrained_embeddings = TEXT.vocab.vectors

UNK_IDX = TEXT.vocab.stoi[TEXT.unk_token]



model = LSTMAutoencoder(INPUT_DIM, 
            EMBEDDING_DIM, 
            HIDDEN_DIM, 
            LATENT_DIM, 
            N_LAYERS, 
            BIDIRECTIONAL, 
            DROPOUT, 
            PAD_IDX,
            TEXT.vocab.vectors)


# Train the model
total_step = len(full_data_iterator)
num_epochs = 1

learning_rate = .0005

# Loss and optimizer
# optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate, momentum=.9)
optimizer = torch.optim.Adam(
    model.parameters(), lr=learning_rate, weight_decay=1e-5)

criterion = nn.MSELoss()


embedding_layer = nn.Embedding(INPUT_DIM, EMBEDDING_DIM, padding_idx = PAD_IDX)
embedding_layer.weight = nn.Parameter(TEXT.vocab.vectors, requires_grad=False)


embedding_layer.weight.data[UNK_IDX] = torch.zeros(EMBEDDING_DIM)
embedding_layer.weight.data[PAD_IDX] = torch.zeros(EMBEDDING_DIM)

embedding_layer.to(device)




def word_ids_to_sentence(id_tensor, vocab, join=None):
    """Converts a sequence of word ids to a sentence"""
    if isinstance(id_tensor, torch.LongTensor):
        ids = id_tensor.transpose(0, 1).contiguous().view(-1)
    elif isinstance(id_tensor, np.ndarray):
        ids = id_tensor.transpose().reshape(-1)
    batch = [vocab.itos[ind] for ind in ids] # denumericalize
    if join is None:
        return batch
    else:
        return join.join(batch)

i = 0
# for epoch in range(num_epochs):
for batch in full_data_iterator:
    optimizer.zero_grad()
    text, text_lengths = batch.Description
    text = text.to(device)
    text_lengths = text_lengths.to(device)

    print(text.shape, "text shape")

    print(word_ids_to_sentence(text[75].cpu().data.numpy(), TEXT.vocab))

    embedded = embedding_layer(text)
    # print(embedded.shape, "text embedding shape")
    # packed_embedded = nn.utils.rnn.pack_padded_sequence(embedded, text_lengths)

    # print(packed_embedded)
    
    predictions = model(embedded)

    # predictions = nn.utils.rnn.pad_packed_sequence(predictions)

    print(predictions.shape, "predictions shape")

    predictions2 = predictions.transpose(1,0)
    print(predictions2.shape, "prediction2s shape")  

    print(predictions2[0][:][1])

    # print(text.shape, "text shape")
    loss = criterion(predictions, embedded)
    
    # Backward and optimize
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    i+=1
    break 
    print ('Batch [{}], Loss: {:.4f}' 
            .format(i, loss.item()))

torch.Size([90, 64]) text shape
['the', 'sale', 'as', 'applied', 'L.', 'it', '"', 'lvandyke@balboa.eng.uci.edu', 'hell', 'have', 'khan0095@nova.gmi.edu', '3', 'that', 'those', 'starts', '?', ',', 'Geo', 'It', 'is', 'is', 'work', '?', 'be', 'resolution', '....', 'never', 'deposit', 'rest', '\n   ', 'Sorry', ',', 'for', 'Thanks', 'Engineering', '?', '.', 'monitor', 'o', '...', ')', "'", '\n\n', '....', 'Never', '\n', 'appreciated', 'Richard', 'Thanks', '\n\n', 'advance', '\n\n\n', '.', '!', '\n\n\n\n\n\t', ' ', '\n\n', '\n', 'Ron', 'George', 'then', ')', '.', '<pad>']
torch.Size([90, 64, 300]) predictions shape
torch.Size([64, 90, 300]) prediction2s shape
tensor([-0.0240, -0.0155, -0.0138, -0.0453, -0.0203, -0.0279,  0.0005, -0.0064,
        -0.0359,  0.0103,  0.0248,  0.0025, -0.0019,  0.0049, -0.0339,  0.0279,
         0.0011,  0.0019, -0.0181, -0.0179,  0.0199,  0.0064, -0.0050,  0.0017,
         0.0176,  0.0071, -0.0221, -0.0138, -0.0152,  0.0047,  0.0014,  0.0157,
        -0.0162,  

In [0]:
import torch.nn as nn
import torch.nn.functional as F


class LSTMAutoencoder(nn.Module):
    def __init__(self, vocab_size, embedding_dim, hidden_dim, latent_dim, n_layers, 
                 bidirectional, dropout, pad_idx, weights):
        
        super().__init__()

        self.hidden_dim = hidden_dim
        self.latent_dim = latent_dim
        
        self.embedding = nn.Embedding(vocab_size, embedding_dim, padding_idx = pad_idx)

        self.embedding.weight = nn.Parameter(weights, requires_grad=False)
        
        self.rnn_encoder = nn.LSTM(embedding_dim, 
                           latent_dim, 
                           num_layers=n_layers, 
                           bidirectional=bidirectional, 
                           dropout=dropout)
        
        # self.fc_encoder = nn.Linear(hidden_dim * 2, latent_dim)

        # self.fc_decoder = nn.Linear(latent_dim, hidden_dim * 2)

        self.rnn_decoder = nn.LSTM(latent_dim*2,embedding_dim, 
                           num_layers=n_layers, 
                           bidirectional=bidirectional, 
                           dropout=dropout)
        
        # self.dropout = nn.Dropout(dropout)
        
    def forward(self, text, text_lengths):
        
        print(text.shape, "input text shape")
        # embedded = self.dropout(self.embedding(text))
        embedded = self.embedding(text)
        print(embedded.shape, "text embedding shape")
        packed_embedded = nn.utils.rnn.pack_padded_sequence(embedded, text_lengths)
        _, (hidden, _) = self.rnn_encoder(packed_embedded)
        print(hidden.shape , "hidden shap after encoder rnn")
        # _, _ = nn.utils.rnn.pad_packed_sequence(packed_output)

        # hidden = self.dropout(torch.cat((hidden[-2,:,:], hidden[-1,:,:]), dim = 1))

        hidden = torch.cat((hidden[-2,:,:], hidden[-1,:,:]), dim = 1)
        print(hidden.shape, "hidden shape after concat")

        hidden = hidden.unsqueeze(0)
                
        #hidden = [batch size, hid dim * num directions]

        # encoded = F.relu(self.fc_encoder(hidden))
        # print(encoded.shape , "encoder output shape")

        # decoded_fc = F.relu(self.fc_decoder(encoded))
        # print(decoded_fc.shape)

        encoded_repeat = hidden.repeat(self.latent_dim,1,1)
        print(encoded_repeat.shape, "encode repeat shape")

        decoded, _ = self.rnn_decoder(encoded_repeat)
        

        print(decoded.shape, " decoded output")
            
        return decoded

INPUT_DIM = len(TEXT.vocab)
EMBEDDING_DIM = 300
HIDDEN_DIM = 256
LATENT_DIM = 10
N_LAYERS = 2
BIDIRECTIONAL = True
DROPOUT = 0
PAD_IDX = TEXT.vocab.stoi[TEXT.pad_token]

pretrained_embeddings = TEXT.vocab.vectors

UNK_IDX = TEXT.vocab.stoi[TEXT.unk_token]



model = LSTMAutoencoder(INPUT_DIM, 
            EMBEDDING_DIM, 
            HIDDEN_DIM, 
            LATENT_DIM, 
            N_LAYERS, 
            BIDIRECTIONAL, 
            DROPOUT, 
            PAD_IDX,
            TEXT.vocab.vectors)

model.embedding.weight.data[UNK_IDX] = torch.zeros(EMBEDDING_DIM)
model.embedding.weight.data[PAD_IDX] = torch.zeros(EMBEDDING_DIM)

# Train the model
total_step = len(full_data_iterator)
num_epochs = 1

learning_rate = .0005

# Loss and optimizer
# optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate, momentum=.9)
optimizer = torch.optim.Adam(
    model.parameters(), lr=learning_rate, weight_decay=1e-5)

criterion = nn.MSELoss()

for epoch in range(num_epochs):
    for batch in full_data_iterator:
        optimizer.zero_grad()
        text, text_lengths = batch.Description
        text = text.to(device)
        text_lengths = text_lengths.to(device)
        
        predictions = model(text, text_lengths).squeeze(1)

        print(predictions.shape)
        print(text.shape)
        loss = criterion(predictions, text)
        
        # Backward and optimize
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        if (i+1) % 100 == 0:
            print ('Epoch [{}/{}], Step [{}/{}], Loss: {:.4f}' 
                   .format(epoch+1, num_epochs, i+1, total_step, loss.item()))

TypeError: ignored